In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug  5 00:08:06 2016

@author: Israel
"""
import numpy as np
import matplotlib.pylab as plt
states = ('Healthy', 'Fever')
end_state = 'E'
 
observations = ('cold', 'normal', 'normal')
 
start_probability = {'Healthy': 0.6, 'Fever': 0.4}

transition_probability = {
   'Healthy' : {'Healthy': 0.69, 'Fever': 0.3, 'E': 0.01},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.59, 'E': 0.01},
   }
 
emission_probability = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
   }
def fwd_bkw(y, states, a_0, a, e, end_st):
    L = len(y)
 
    fwd = []
    f_prev = {}
    # forward part of the algorithm
    for i, y_i in enumerate(y):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = a_0[st]
            else:
                prev_f_sum = sum(f_prev[k]*a[k][st] for k in states)
 
            f_curr[st] = e[st][y_i] * prev_f_sum
 
        fwd.append(f_curr)
        f_prev = f_curr
 
    p_fwd = sum(f_curr[k]*a[k][end_st] for k in states)
 
    bkw = []
    b_prev = {}
    # backward part of the algorithm
    for i, y_i_plus in enumerate(reversed(y[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = a[st][end_st]
            else:
                b_curr[st] = sum(a[st][l]*e[l][y_i_plus]*b_prev[l] for l in states)
 
        bkw.insert(0,b_curr)
        b_prev = b_curr
 
    p_bkw = sum(a_0[l] * e[l][y[0]] * b_curr[l] for l in states)
 
    # merging the two parts
    posterior = []
    for i in range(L):
        posterior.append({st: fwd[i][st]*bkw[i][st]/p_fwd for st in states})
 
    assert p_fwd == p_bkw
    return fwd, bkw, posterior
def example():
    return fwd_bkw(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability,
                   end_state)
 
fwd, bkw, posterior = example()
print(posterior)
import random
states = ('Healthy', 'Fever')
observations = ('normal', 'cold', 'dizzy')
start_probability = {'Healthy': 0.6, 'Fever': 0.4}

transition_probability = {
   'Healthy' : {'Healthy': 0.8, 'Fever': 0.2},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
   }

emission_probability = {
   'Healthy' : {'normal': 0.7, 'cold': 0.2, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.4, 'dizzy': 0.5}
   }

# A random HMM is generated using the probability matricies defined above
# Both the latent and visible states are generated in order to later assess the accuracy of our algorithms
N = 100
hidden = []
visible = []
# generate observations
if random.random() < start_probability[states[0]]:
    hidden.append(states[0])
else:
    hidden.append(states[1])

for i in range(N):
    current_state = hidden[i]
    if random.random() < transition_probability[current_state][states[0]]:
        hidden.append(states[0])
    else:
        hidden.append(states[1])
    r = random.random()
    prev = 0
    for observation in observations:
        prev += emission_probability[current_state][observation]
        if r < prev:
            visible.append(observation)
            break

# fixes issue of creating extra hidden state
hidden.pop()
def print_dptable(V):
    s = "    " + " ".join(("%7d" % i) for i in range(len(V))) + "\n"
    for y in V[0]:
        s += "%.5s: " % y
        s += " ".join("%.7s" % ("%f" % v[y]) for v in V)
        s += "\n"
    print(s)

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}
 
    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]
 
    # Run Viterbi for t > 0
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
 
        for y in states:
            (prob, state) = max((V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states)
            V[t][y] = prob
            newpath[y] = path[state] + [y]
 
        # Don't need to remember the old paths
        path = newpath
     
    #print_dptable(V)
    (prob, state) = max((V[t][y], y) for y in states)
    return (prob, path[state])

# input the generated markov model
def example():
    return viterbi(visible,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability)

(prob, p_hidden) = example()

# assess accuracy of the HMM model
wrong= 0
for i in range(len(hidden)):
    if hidden[i] != p_hidden[i]:
        wrong = wrong + 1
print("accuracy: " + str(1-float(wrong)/N))

[{'Healthy': 0.7523637371047066, 'Fever': 0.24763626289529342}, {'Healthy': 0.9154595401170142, 'Fever': 0.08454045988298578}, {'Healthy': 0.907504985311503, 'Fever': 0.09249501468849691}]
accuracy: 0.81


# HMM implementation for the data found in
#https://www.12000.org/my_notes/HMM_program/report/report.htm#x1-60003

len <- 5; 
states<- c("S", "T")
observations<-c("A","C","T")
#-----------------------------------------------------------
#function to calculate transition matrix
#-----------------------------------------------------------
tm <- matrix(data=0, ncol=2, nrow=2);
rownames(tm)<-states
colnames(tm)<-states

#from S
tm[1,1] <- 0.7;tm[1,2] <- 0.3;
#from T
tm[2,1] <- 0.4;  tm[2,2] <- 0.6;
#tm[2,1] <- 0.3;  tm[2,2] <- 0.7
tm

#-----------------------------------------------------------
#function to calculate emission probabilities
#-----------------------------------------------------------

em <- matrix(0, nrow=2, ncol=3);
rownames(em)<-states
colnames(em)<-observations

#em[1,1] <- 0.5; em[1,2] <- 0.4;  em[1,3] <- 0.1;
#em[2,1] <- 0.1; em[2,2] <- 0.3; em[2,3] <- 0.6;
em[1,1] <- 0.4; em[1,2] <- 0.4;  em[1,3] <- 0.2;
em[2,1] <- 0.25; em[2,2] <- 0.55; em[2,3] <- 0.2;
em


#-----------------------------------------------------------
#prepare storage
sta <- numeric(len);
obs <- numeric(len);


#choose random starting position
#prior <- c(0.6, 0.4);
prior <- c(0.4, 0.6);
sta[1] <- sample(1:2, 1, prob=prior);
obs[1] <- sample(1:3, 1, prob=em[sta[1],]);


#run MC
for(i in 2:len){
  sta[i] <- sample(1:2, 1, prob=tm[sta[i-1],]);
  obs[i] <- sample(1:3, 1, prob=em[sta[i],]);
}
#return
data.frame(sta=sta, obs=obs);

#-----------------------------------------------------------
#Function to calculate LL
#-----------------------------------------------------------
forward <- matrix(data=0, nrow=2, ncol=len);

forward[,1] <- em[,obs[1]] * prior;
s <- sum(forward[,1]);
forward[,1] <- forward[,1] / s;
scale <- log10(s);

#now make recursion    
for(i in 2:len){
  #swap entries
  #calc forward
  forward[,i] <- (forward[,i-1] %*% tm) * em[,obs[i]];
  
  s <- sum(forward[,i]);
  forward[,i] <- forward[,i] / s; #calculates the probability of being in state si at time t given observing observation o1to ot inclusive
  #p(St=si,O1:t(forward[,i] or alpha)/s(sum over all the forward prob at time i)
  scale <- scale + log10(s);    
}
list("forward"= forward, "scale"=scale, "em"=em, "tm"=tm)

backward <- matrix(data=0, nrow=2, ncol=len);
gamma<-matrix(data=0, nrow=2, ncol=len)

#Start with initial  

backward[,len] <- 1;  
#now make recursion    
for(i in (len-1):1){
  
  #calc backward
  backward[,i] <- (backward[,i+1]*em[,obs[i+1]])%*% tm ;
  s=sum(backward[,i])
  backward[,i] <- backward[,i] / s;
  gamma[,i]<-forward[,i]*backward[,i]
}

list("forward"= forward, "backward"=backward, "gamma"=gamma, "scale"=scale, "em"=em, "tm"=tm)


